# Object Detection - SSD Mobilenet V2 from TensorFlow Hub (Image URLs)

This model identifies objects present in images, returning its scores, labels and bounding boxes.

We are using a dataset from [UCF](https://www.crcv.ucf.edu/data/GMCP_Geolocalization/#Dataset) and the model [SSD Mobilenet V2](https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1) from [TensorFlow Hub](https://www.tensorflow.org/hub).

<a href="https://colab.research.google.com/github/VertaAI/examples/blob/main/deployment/tfhub/object-detection/url/object-detection-url-predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dependencies

This notebook has been tested with **Python 3.8.15** and the following package versions:

In [ ]:
%%capture
!pip install beautifulsoup4==4.6.3
!pip install requests==2.28.1
!pip install verta==0.21.1

## Imports

In [ ]:
import concurrent.futures
import os
import requests
import time

from bs4 import BeautifulSoup
from verta import Client

## Verta Set Up

In [ ]:
os.environ['VERTA_HOST'] = ''
os.environ['VERTA_EMAIL'] = ''
os.environ['VERTA_DEV_KEY'] = ''

In [ ]:
client = Client(os.environ['VERTA_HOST'])

In [ ]:
endpoint = client.get_endpoint('object-detection-url')

In [ ]:
model = endpoint.get_deployed_model()

## Get URLs

In [ ]:
url = 'http://www.cs.ucf.edu/~aroshan/index_files/Dataset_PitOrlManh/images/'

In [ ]:
req = requests.get(url)

In [ ]:
soup = BeautifulSoup(req.text, 'lxml')

In [ ]:
urls = []

for link in soup.find_all('a'):
    href = link.get('href')
  
    if href.endswith('.jpg'):
      url = f"http://www.cs.ucf.edu/~aroshan/index_files/Dataset_PitOrlManh/images/{href}"
      urls.append(url)

## Show Process Metrics

In [ ]:
def show_metrics(urls, start_time, end_time):
    total_time = end_time - start_time
    total_time = time.strftime('%Hh %Mm %Ss', time.gmtime(total_time))
    
    print(f"Processing Time: {total_time} for {len(urls)} URLs.")

## Predict

In [ ]:
def process_image(data):
    return model.predict(data)

In [ ]:
# urls = urls[:100]
# urls = urls[:1000]
urls = urls[:10000]

In [ ]:
data = [[os.path.basename(url), url] for url in urls]

In [ ]:
results = []
start_time = time.time()

with concurrent.futures.ThreadPoolExecutor() as executor:
    for result in zip(data, executor.map(process_image, data)):
        results.append(result[1])

end_time = time.time()

In [ ]:
show_metrics(urls, start_time, end_time)